# Working with Trajectories

In this example you will see how you can use QMzyme to generate multiple QMzyme Models for different trajectory snapshots. 

Classes used in this example:

- [GenerateModel](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.GenerateModel.html)

- [QM_Method](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.CalculateModel.html#qm-treatment)

- [XTB_Method](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.CalculateModel.html#xtb-treatment)

- [CA_terminal TruncationScheme](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.TruncationSchemes.html#QMzyme.TruncationSchemes.CA_terminal)

- [DistanceCutoff SelectionScheme](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.SelectionSchemes.html#QMzyme.SelectionSchemes.DistanceCutoff)


In [1]:
import QMzyme
from QMzyme.SelectionSchemes import DistanceCutoff
from QMzyme.data import PQR, DCD

In [2]:
# How many frames?
model = QMzyme.GenerateModel(PQR, DCD)
print("Number of Frames: ", model.universe.trajectory.n_frames)

Number of Frames:  50


/Users/hrk/anaconda3/envs/qmzyme_py3.10/lib/python3.10/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"
/Users/hrk/git/QMzyme/QMzyme/MDAnalysisWrapper.py:28: UserWarning: Element information was missing from input. MDAnalysis.topology.guessers.guess_types was used to infer element types.
  warnings.warn("Element information was missing from input. MDAnalysis.topology.guessers.guess_types was used to infer element types.", UserWarning)


#### QM-only: Loop over frames to generate QMzymeModels
In this first scenario, you will generate five QM-only models using the DistanceCutoff selection scheme with a cutoff of 3 Angstroms.

In [3]:
%%time

qm_method = QMzyme.QM_Method(
    basis_set='6-31G*', 
    functional='wB97X-D3', 
    qm_input='OPT FREQ', 
    program='orca'
)

for frame in range(0, 50, 10):
    print('\n====================================')
    print(f'             Frame {frame}')
    print('====================================')
    m = QMzyme.GenerateModel(PQR, DCD, frame=frame)
    m.set_catalytic_center('resid 263')
    m.set_region(selection=DistanceCutoff, cutoff=3)
    m.cutoff_3.rename(f'{m.cutoff_3.name}_frame_{frame}')
    c_alpha_atoms = m.regions[-1].get_atoms(attribute='name', value='CA')
    m.regions[-1].set_fixed_atoms(atoms=c_alpha_atoms)
    qm_method.assign_to_region(region=m.regions[-1])
    m.truncate()
    m.write_input(filename=f"{m.name}_cutoff3_frame{frame}_qm")


             Frame 0

Calculating total charge for QMzymeRegion cutoff_3_frame_0 based on charges read from topology attribute 'charge'...
	<QMzymeResidue resname: TYR, resid: 16, chain: X> --> Charge: 0
	<QMzymeResidue resname: VAL, resid: 20, chain: X> --> Charge: 0
	<QMzymeResidue resname: TYR, resid: 57, chain: X> --> Charge: 0
	<QMzymeResidue resname: GLY, resid: 60, chain: X> --> Charge: 0
	<QMzymeResidue resname: LEU, resid: 61, chain: X> --> Charge: 0
	<QMzymeResidue resname: VAL, resid: 66, chain: X> --> Charge: 0
	<QMzymeResidue resname: PHE, resid: 86, chain: X> --> Charge: 0
	<QMzymeResidue resname: VAL, resid: 88, chain: X> --> Charge: 0
	<QMzymeResidue resname: MET, resid: 90, chain: X> --> Charge: 0
	<QMzymeResidue resname: TRP, resid: 92, chain: X> --> Charge: 0
	<QMzymeResidue resname: LEU, resid: 99, chain: X> --> Charge: 0
	<QMzymeResidue resname: VAL, resid: 101, chain: X> --> Charge: 0
	<QMzymeResidue resname: ASH, resid: 103, chain: X> --> Charge: 0
	<QMzymeResid

#### QM/xTB: Loop over frames to generate QMzymeModels
In this second scenario, you will generate five QM/xTB models with a QM region consisting of the ligand, EQU and the catalytic ASP103 residue and a large xTB region containing residues within 8 Angstroms of EQU.

In [3]:
%%time

qm_method = QMzyme.QM_Method(
    basis_set='6-31+G*', 
    functional='wB97X-D3', 
    qm_input='OPT FREQ', 
    program='orca'
)

for frame in range(0, 50, 10):
    print('\n====================================')
    print(f'             Frame {frame}')
    print('====================================')
    m = QMzyme.GenerateModel(PQR, DCD, frame=frame)
    m.set_catalytic_center('resid 263')
    m.set_region(name='qm_region', selection='resid 263 or resid 103')
    m.set_region(selection=DistanceCutoff, cutoff=8)
    m.cutoff_8.rename(f'{m.cutoff_8.name}_frame_{frame}')
    qm_method.assign_to_region(region=m.qm_region)
    QMzyme.XTB_Method().assign_to_region(region=m.cutoff_8)
    m.truncate()
    m.pymol_visualize(filename=f"1oh0_qmxtb_frame{frame}.py")
    m.write_input(filename=f"{m.name}_cutoff8_frame{frame}_qmxtb")


             Frame 0

Calculating total charge for QMzymeRegion qm_region based on charges read from topology attribute 'charge'...
	<QMzymeResidue resname: ASH, resid: 103, chain: X> --> Charge: 0
	<QMzymeResidue resname: EQU, resid: 263, chain: X> --> Charge: -1
QMzymeRegion qm_region has a total charge of -1.

Calculating total charge for QMzymeRegion cutoff_8_frame_0 based on charges read from topology attribute 'charge'...
	<QMzymeResidue resname: TYR, resid: 16, chain: X> --> Charge: 0
	<QMzymeResidue resname: ILE, resid: 17, chain: X> --> Charge: 0
	<QMzymeResidue resname: LEU, resid: 19, chain: X> --> Charge: 0
	<QMzymeResidue resname: VAL, resid: 20, chain: X> --> Charge: 0
	<QMzymeResidue resname: ASP, resid: 21, chain: X> --> Charge: -1
	<QMzymeResidue resname: GLY, resid: 23, chain: X> --> Charge: 0
	<QMzymeResidue resname: ILE, resid: 28, chain: X> --> Charge: 0
	<QMzymeResidue resname: TYR, resid: 32, chain: X> --> Charge: 0
	<QMzymeResidue resname: VAL, resid: 38, chain